In [8]:
#code taken from https://github.com/madelinehayes/seabirdNET/blob/master/via_to_retinanet_FINAL.ipynb

In [22]:
import os
import argparse
import numpy as np
import json
import csv

In [23]:
import os
path = '../Images/Harbor_Seals_Tile'
image_list = [f for f in os.listdir(path) if f.endswith('.png')]
#image_list

In [24]:
#shuffle image list and split into training, testing, validation
import random

random.shuffle(image_list)
total_count = len(image_list)

#print(total_count)

test_index = int(total_count*.1)
val_index = int(total_count*.9*.1) + test_index

#print(test_index_albatross, val_index_albatross)

test_data = image_list[:test_index]
val_data = image_list[test_index:val_index]
train_data = image_list[val_index:]

print(len(test_data), len(val_data), len(train_data))

308 277 2495


In [25]:
via_path = '../Data/Tiled_Harbor_Seals.csv'

image_annotations_train = []
image_annotations_test = []
image_annotations_val = []

with open(via_path, "r") as f:
    reader = csv.reader(f, delimiter=",")
    for line in reader: 
        # output we want
        # path/to/image.jpg,x1,y1,x2,y2,class_name
        # /data/imgs/img_001.jpg,837,346,981,456,cow
        if '#' in line[0][0]:
            # bypassing comments in csv
            continue
        filename = line[1][2:-2]
        # strip brackets, split and get only the values we care about, then convert all the string to int 
        coords = list(map(int,list(map(float, line[4].strip('][').split(',')[1:]))))
        if len(coords) != 4:
            print('bad annotation showed up')
            continue
            
        top_left_x, top_left_y, width, height = list(map(int,list(map(float, line[4].strip('][').split(',')[1:]))))

        if width == 0 or height == 0:
            continue
        # move from top left and width/height to x and y values
        if top_left_x < 0:
            top_left_x = 1
        if top_left_y < 0:
            top_left_y = 1
        x1 = top_left_x
        x2 = top_left_x + width
        y1 = top_left_y
        y2 = top_left_y + height 

        name = "seal"

        # create the csv row
        new_row = []
        if filename in train_data:
            new_row.append(filename)
            new_row.append(x1)
            new_row.append(y1)
            new_row.append(x2)
            new_row.append(y2)
            new_row.append(name)

            image_annotations_train.append(new_row)
        elif filename in test_data:
            new_row.append(filename)
            new_row.append(x1)
            new_row.append(y1)
            new_row.append(x2)
            new_row.append(y2)
            new_row.append(name)

            image_annotations_test.append(new_row)
        else:
            new_row.append(filename)
            new_row.append(x1)
            new_row.append(y1)
            new_row.append(x2)
            new_row.append(y2)
            new_row.append(name)

            image_annotations_val.append(new_row)

In [21]:
with open('../Data/harbor_test_annotations.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_test)
    
with open('../Data/harbor_val_annotations.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_val)
    
with open('../Data/harbor_train_annotations.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_train)

In [17]:
#not sure what this is and where it should go but... come back
#'mosaics/tiles/classes.csv'
detection_classes = [["seal", 0]]
with open('../Data/harbor_classes.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(detection_classes)

In [26]:
print(len(image_annotations_test))

198


In [27]:
print(len(image_annotations_val))

168


In [28]:
print(len(image_annotations_train))

1298


In [29]:
%run "../keras-retinanet/keras_retinanet/bin/evaluate.py" csv ../Images/Harbor_Seals_Tile/harbor_train_annotations.csv ../Images/Harbor_Seals_Tile/harbor_classes.csv ../Data/final_model_harbor_seal.h5

Using TensorFlow backend.


Loading model, this may take a second...


Running network: 100% (158 of 158) |#####| Elapsed Time: 0:11:40 Time:  0:11:40
Parsing annotations: 100% (158 of 158) |#| Elapsed Time: 0:00:00 Time:  0:00:00


1436 instances of class seal with average precision: 0.6338
Inference time for 158 images: 4.2920
mAP using the weighted average of precisions among classes: 0.6338
mAP: 0.6338
